# Day 2a: Agent Tools（LangChain可本地运行版）

本教程是用 LangChain 框架复现 Kaggle 课程 "Day 2: Tools" 的内容。我们将学习如何为 Agent 装备"双手"（工具）和"大脑"（代码执行）。

## 🎯 学习目标
在完成本教程后，你将能够：
1. ✅ **创建自定义工具**: 将 Python 函数转换为 Agent 可用的工具。
2. ✅ **实现代码执行**: 让 Agent 编写并运行 Python 代码以进行精确计算。
3. ✅ **理解 Agent 能力**: 明白为什么工具对于 Agent 连接真实世界至关重要。

**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。

## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community python-dotenv

### 1.3 导入依赖库

In [2]:
import os
import sys
import json
from dotenv import load_dotenv, find_dotenv

# LangChain 组件
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field

# 忽略警告
import warnings
from langchain_core._api.deprecation import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
warnings.filterwarnings("ignore")

print("✅ 依赖库加载完成")

✅ 依赖库加载完成


### 1.4 配置 API Key

> 关于 .env 文件内容说明，请查看 [README 环境配置文件 (.env) 说明](../README.md#3-环境配置文件-env-说明)

In [ ]:
# 加载环境变量
load_dotenv(find_dotenv())

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get("MODEL_NAME")
    api_key = os.environ.get("OPENAI_API_KEY")
    base_url = os.environ.get("OPENAI_API_BASE")

    if not api_key:
        print("❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY")
        return None
    
    # 打印部分 Key 以验证 (隐藏敏感信息)
    masked_key = f"{api_key[:8]}..." if api_key else "None"
    print(f"✅ API Key 已加载: {masked_key}")
    print(f"✅ 使用模型: {model_name}")
    if base_url:
        print(f"✅ API Base URL: {base_url}")
    
    return {
        "model_name": model_name,
        "api_key": api_key,
        "base_url": base_url,
        "extra_body": json.loads(os.environ.get("EXTRA_BODY", "{}"))
    }

config = configure_api_keys()

✅ API Key 已加载: ms-04d57...
✅ 使用模型: Qwen/Qwen3-235B-A22B-Instruct-2507
✅ API Base URL: https://api-inference.modelscope.cn/v1
✅ Tavily Search API 已启用 (增强搜索能力)


### 1.5 初始化 LLM

现在，让我们初始化 LangChain 的 ChatOpenAI 模型。这相当于 ADK 中的 `Gemini` 模型配置。

In [4]:
# 初始化 LLM
llm = ChatOpenAI(
    model=config["model_name"],
    api_key=config["api_key"],
    base_url=config["base_url"],
    temperature=0.1, # 低温度以保证工具调用稳定
    extra_body=config["extra_body"]
)

print("✅ LLM 初始化完成")

✅ LLM 初始化完成


## 🤖 2. 什么是自定义工具 (Custom Tools)?

如果没有工具，Agent 的知识就被冻结在它训练结束的那一刻。它无法访问今天的汇率，也无法查询你的库存。
**解决方案**: 自定义工具就是你写的 Python 函数。通过简单的装饰器，我们就能把任何 Python 代码变成 Agent 的"双手"。

### 2.1 定义工具
我们将创建两个工具来模拟货币兑换场景：
1. `get_fee_for_payment_method`: 查询手续费。
2. `get_exchange_rate`: 查询汇率。


In [5]:
class PaymentMethodInput(BaseModel):
    method: str = Field(description="The name of the payment method, e.g., 'platinum credit card' or 'bank transfer'")

@tool("get_fee_for_payment_method", args_schema=PaymentMethodInput)
def get_fee_for_payment_method(method: str) -> dict:
    """查询指定支付方式的交易手续费百分比。
    该工具模拟根据用户提供的支付方式名称，查找公司内部的费用结构。
    """
    print(f"    🔧 [Tool Call] 查询手续费: '{method}' ")
    
    # 模拟数据库
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,     # 1%
    }

    if not isinstance(method, str):
        return {"status": "error", "error_message": f"Invalid input type: {type(method)}"}

    fee = fee_database.get(method.lower())
    if fee is not None:
        result = {"status": "success", "fee_percentage": fee}
        print(f"    ✅ [Tool Return] 找到费率: {fee*100}%")
        return result
    else:
        result = {"status": "error", "error_message": f"未找到支付方式 '{method}'"}
        print(f"    ❌ [Tool Return] {result['error_message']}")
        return result

class ExchangeRateInput(BaseModel):
    base_currency: str = Field(description="The ISO 4217 currency code of the currency you are converting from (e.g., 'USD').")
    target_currency: str = Field(description="The ISO 4217 currency code of the currency you are converting to (e.g., 'EUR').")

@tool("get_exchange_rate", args_schema=ExchangeRateInput)
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies."""
    print(f"    🔧 [Tool Call] 查询汇率: {base_currency} -> {target_currency}")

    # 模拟实时汇率 API
    rate_database = {
        "usd": {
            "eur": 0.93,
            "jpy": 157.50,
            "inr": 83.58,
            "cny": 7.25,
        }
    }

    try:
        base = str(base_currency).lower().strip()
        target = str(target_currency).lower().strip()
        
        base_rates = rate_database.get(base, {})
        rate = base_rates.get(target)
        
        if rate is not None:
            result = {"status": "success", "rate": rate}
            print(f"    ✅ [Tool Return] 找到汇率: 1 {base.upper()} = {rate} {target.upper()}")
            return result
        else:
            result = {"status": "error", "error_message": f"不支持的货币对: {base_currency}/{target_currency}"}
            print(f"    ❌ [Tool Return] {result['error_message']}")
            return result
    except Exception as e:
        print(f"    ❌ [Tool Error] {e}")
        return {"status": "error", "error_message": str(e)}

print("✅ 工具函数已定义")

✅ 工具函数已定义


### 2.2 创建并运行基础 Agent

我们将使用 `create_tool_calling_agent` 创建一个具备上述两个工具的 Agent，并询问它一个涉及汇率转换的问题。


In [6]:
# 1. 创建 Agent
currency_tools = [get_fee_for_payment_method, get_exchange_rate]

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method` to find transaction fees
    2. Use `get_exchange_rate` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.

    Please always respond in Chinese.
"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, currency_tools, prompt)
runner = AgentExecutor(agent=agent, tools=currency_tools, verbose=False, handle_parsing_errors=True)

# 2. 运行演示
query = "我想用我的白金信用卡把 500 美元换成欧元，我会收到多少钱？"
print(f"用户提问: {query}\n")

try:
    response = runner.invoke({"input": query})
    # Use triple quotes for multi-line f-strings to avoid SyntaxError
    print(f"""\n🔮 Agent 回答:\n{response['output']}""")
except Exception as e:
    print(f"❌ 运行出错: {e}")


用户提问: 我想用我的白金信用卡把 500 美元换成欧元，我会收到多少钱？

    🔧 [Tool Call] 查询手续费: 'platinum credit card' 
    ✅ [Tool Return] 找到费率: 2.0%
    🔧 [Tool Call] 查询汇率: USD -> EUR
    ✅ [Tool Return] 找到汇率: 1 USD = 0.93 EUR

🔮 Agent 回答:
您将收到 455.70 欧元。

计算过程如下：

1. **交易手续费**：使用白金信用卡的手续费为 2%。  
   手续费金额 = 500 美元 × 2% = 10 美元。

2. **扣除手续费后的金额**：  
   500 美元 - 10 美元 = 490 美元。

3. **汇率转换**：当前美元兑欧元的汇率为 0.93。  
   转换后金额 = 490 美元 × 0.93 = 455.70 欧元。

因此，最终您将收到 455.70 欧元。


## 💻 3. 用代码增强 Agent (Code Execution)

**问题**: LLM 并不擅长复杂的数学计算，经常产生"幻觉"（比如算出 1+1=3）。
**解决方案**: 我们不再让 Agent 自己心算，而是给它一个**Python 代码执行工具**，让它写代码来计算。


In [7]:
@tool
def execute_python_code(code: str) -> str:
    """执行 Python 代码并返回打印的输出。
    
    该工具接收 Python 代码字符串，在受限的环境中执行它，并返回标准输出(stdout)。
    请务必使用 print() 函数打印你需要查看的所有结果。
    """
    print(f"    🐍 [Code Executor] 正在执行 Python 代码...")
    
    # 打印代码供查看
    for i, line in enumerate(code.strip().split('\n'), 1):
        print(f"      {i:02d}| {line}")
    
    import io
    import contextlib
    
    stdout_capture = io.StringIO()
    try:
        # 定义受限的全局命名空间
        safe_globals = {"__builtins__": {"print": print, "range": range, "len": len, "int": int, "float": float, "str": str, "round": round, "abs": abs, "min": min, "max": max, "sum": sum}}
        
        with contextlib.redirect_stdout(stdout_capture):
            exec(code, safe_globals)
        
        output = stdout_capture.getvalue()
        if not output: output = "Code executed successfully but printed nothing."
            
        print(f"""    📤 [Code Result] 执行结果:\n{output}""")
        return output
    except Exception as e:
        return f"Execution Error: {type(e).__name__}: {str(e)}"

print("✅ 代码执行工具 `execute_python_code` 已定义")

✅ 代码执行工具 `execute_python_code` 已定义


### 3.1 运行增强版 Agent

现在创建一个包含 `execute_python_code` 工具的新 Agent，并要求它进行精确计算。我们明确指示它**必须写代码**而不能心算。


In [8]:
enhanced_tools = [get_fee_for_payment_method, get_exchange_rate, execute_python_code]

enhanced_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the execute_python_code tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
           
   Please always respond in Chinese.
"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

enhanced_agent = create_tool_calling_agent(llm, enhanced_tools, enhanced_prompt)
enhanced_runner = AgentExecutor(agent=enhanced_agent, tools=enhanced_tools, verbose=False, handle_parsing_errors=True)

query2 = "通过银行转账将 1250 美元兑换成印度卢比 (INR)。请展示精确计算。"
print(f"用户提问: {query2}\n")

try:
    response2 = enhanced_runner.invoke({"input": query2})
    # Use triple quotes for multi-line f-strings to avoid SyntaxError
    print(f"""\n🔮 Enhanced Agent 回答:\n{response2['output']}""")
except Exception as e:
    print(f"❌ 运行出错: {e}")


用户提问: 通过银行转账将 1250 美元兑换成印度卢比 (INR)。请展示精确计算。

    🔧 [Tool Call] 查询手续费: 'bank transfer' 
    ✅ [Tool Return] 找到费率: 1.0%
    🔧 [Tool Call] 查询汇率: USD -> INR
    ✅ [Tool Return] 找到汇率: 1 USD = 83.58 INR
    🐍 [Code Executor] 正在执行 Python 代码...
      01| fee_percentage = 0.01
      02| exchange_rate = 83.58
      03| amount_usd = 1250
      04| 
      05| # 计算手续费金额
      06| fee_amount = amount_usd * fee_percentage
      07| # 扣除手续费后的可用金额
      08| amount_after_fee = amount_usd - fee_amount
      09| # 计算最终兑换的印度卢比金额
      10| final_inr_amount = amount_after_fee * exchange_rate
      11| 
      12| print(f"{final_inr_amount}")
    📤 [Code Result] 执行结果:
103430.25


🔮 Enhanced Agent 回答:
通过银行转账将 1250 美元兑换成印度卢比（INR）的最终金额为 **103,430.25 INR**。

计算过程如下：
- 银行转账的交易手续费为 1%。
  - 手续费金额：1250 USD × 1% = 12.50 USD
  - 扣除手续费后的可用金额：1250 USD - 12.50 USD = 1237.50 USD
- 当前美元兑印度卢比的汇率为 1 USD = 83.58 INR。
- 最终兑换金额：1237.50 USD × 83.58 = 103,430.25 INR。


## 🎓 4. 理论总结

通过今天的学习，我们掌握了：
1. **Tool Use (工具使用)**: Agent 的核心能力，使其能与外部世界交互（如查询数据库、API）。
2. **Code Execution (代码执行)**: 一种特殊的工具，让 Agent 具备了逻辑运算和精确计算的能力，弥补了 LLM 的短板。

这构成了 Agent "所思 (Prompt) -> 所行 (Tool) -> 所得 (Result)" 的基本循环。
